Homework 7 <br>
Sam Odle

In [3]:
import torch
from torch.backends import cudnn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
import random
import warnings
import argparse

GPU Implementation

In [2]:
CUDA_DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(CUDA_DEVICE)


print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__CUDA Device Name:',torch.cuda.get_device_name(0))
print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)


cuda
__CUDNN VERSION: 8200
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA GeForce RTX 2080 Ti
__CUDA Device Total Memory [GB]: 11.810832384


Correctly pre-processes images into train, validation, and holdout by randomly specifying indicies and feeding these to torch.utils.data.sampler.SubsetRandomSampler

In [13]:
from torch.utils.data import SubsetRandomSampler
num_workers = 1
pin_memory = True
batch_size = 32
valid_size = 0.1

main_dataset = datasets.ImageFolder(root='images/')

num_images = len(main_dataset)
indices = list(range(num_images))
n_test = int( num_images * .1 )  # number of test/val elements
n_train = num_images - 2 * n_test

np.random.seed(42)
idx = list(range(num_images))  # indices to all elements
random.shuffle(idx)  # in-place shuffle the indices

train_idx = idx[:n_train]
valid_idx = idx[n_train:(n_train + n_test)]
test_idx = idx[(n_train + n_test):]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler= SubsetRandomSampler(test_idx)

train_loader = torch.utils.data.DataLoader(
    main_dataset, batch_size=batch_size, sampler=train_sampler,
    num_workers=num_workers, pin_memory=pin_memory, drop_last=False
)

test_loader = torch.utils.data.DataLoader(
    main_dataset, batch_size=batch_size, sampler=test_sampler,
    num_workers=num_workers, pin_memory=pin_memory, drop_last=False
)

valid_loader = torch.utils.data.DataLoader(
    main_dataset, batch_size=batch_size, sampler=valid_sampler,
    num_workers=num_workers, pin_memory=pin_memory, drop_last=False
)


Correctly adds applicable data augmentations to your DataLoader

In [15]:
# define transforms
valid_transform = transforms.Compose([
        transforms.Resize(size=[224,224]),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])

train_transforms = [
    transforms.RandomRotation(degrees=90),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize(size=[224,224]),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])  # Imagenet standards
]

train_loader.transform= train_transforms
test_loader.transform = valid_transform
valid_loader.transform = valid_transform


Correctly download and instantiate a CNN pre-trained on ImageNet
<br>
Densenet won best paper award CVPR2017

In [16]:
import torchvision
model = torchvision.models.densenet161(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to C:\Users\Samuel/.cache\torch\hub\checkpoints\densenet161-8d451a50.pth
100.0%


Correctly fine-tune that CNN on your blood cell images using training and validation

In [ ]:
num_classes = 2
num_ftrs = model_ft.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

for param in model.parameters():
    param.requires_grad = False

Report your results on the holdout set